# HPO Graph

In [27]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import configparser
import pandas as pd
from util.graphdb_base import GraphDBBase

## Import Configuration File

In [28]:
config_file = "../../config.ini"
config = configparser.ConfigParser()
config.read(config_file)
neo4j_config = config["neo4j"]
uri = neo4j_config.get("uri")
user = neo4j_config.get("user")
password = neo4j_config.get("password")
encrypted = neo4j_config.getboolean("encrypted")
database = neo4j_config.get("database")
print(f"URI: {uri}")
print(f"User: {user}")
print(f"Password: {password}")
print(f"Encrypted: {encrypted}")
print(f"Database: {database}")

URI: bolt://localhost:7687
User: neo4j
Password: password
Encrypted: False
Database: neo4jdoctor


## Read Graph Data

In [29]:
def show_result_in_df(query, gd, pd):
    with gd._driver.session(database=gd._database) as session:
        r = session.run(query)
        return pd.DataFrame.from_dict(r.data())

In [30]:
gd = GraphDBBase()
gd._database = database
query = "MATCH (n) RETURN n LIMIT 10"
with gd._driver.session(database=gd._database) as session:
    r = session.run(query)
    if len(r.data()) == 0:
        raise RuntimeError("No data in the database")
    else:
        print(len(r.data()))

0


In [31]:
query = """
MATCH (phe:Hpo)
WHERE phe.label = "Growth delay"
OR phe.label = "Large knee"
OR phe.label = "Sensorineural hearing impairment"
OR phe.label = "Pruritus"
OR phe.label = "Type I diabetes mellitus"
WITH phe
MATCH path=(dis:Disease)-[:HAS_PHENOTYPIC_FEATURE]->(phe)
UNWIND dis as nodes
RETURN dis.id as disease_id, 
dis.label as disease_name,
collect(phe.label) as features,
count(nodes) as num_of_features
ORDER BY num_of_features DESC, disease_name
LIMIT 5
"""
res = show_result_in_df(query, gd, pd)
res